# Libraries

In [1]:
from datasets import load_dataset, load_from_disk
from nltk.tokenize import sent_tokenize
import nltk
import pandas as pd
import numpy as np

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Data Step

For this process it will be use the dataset with the topic extracted from the Topic Generation model. This model will be created using those topics.

In [3]:
%%capture
#dataset = load_dataset("rjac/all-the-news-2-1-Component-ones-cluster-labels")

In [4]:
#dataset.save_to_disk("dataset")

In [5]:
dataset = load_from_disk("dataset")

The labels will be compose of the Category and the topic (Or subcategory) of the text in order to have composite structure 

In [6]:
#create tables columns concatenating categpry and topic e.g: "Politics" + "Hong Kong" = "Politics.Hong Kong"
dataset = dataset.map(lambda row: {"labels":row["category"] + " - " + row["topic"] },num_proc=8)

Parameter 'function'=<function <lambda> at 0x7fac023e2430> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


#0:   0%|          | 0/309440 [00:00<?, ?ex/s]

#1:   0%|          | 0/309440 [00:00<?, ?ex/s]

#2:   0%|          | 0/309440 [00:00<?, ?ex/s]

#3:   0%|          | 0/309440 [00:00<?, ?ex/s]

#6:   0%|          | 0/309440 [00:00<?, ?ex/s]

#4:   0%|          | 0/309440 [00:00<?, ?ex/s]

#5:   0%|          | 0/309440 [00:00<?, ?ex/s]

#7:   0%|          | 0/309440 [00:00<?, ?ex/s]

In [7]:
dataset["train"][0]

{'paragraph': 'Hong Kong has fallen into recession, hit by more than five months of anti-government protests that show no signs of relenting, and is unlikely to achieve annual economic growth this year, the city\'s Financial Secretary said. "The blow to our economy is comprehensive," Paul Chan said in a blog post on Sunday, adding that a preliminary estimate for third-quarter GDP on Thursday would show two successive quarters of contraction — the technical definition of a recession. He also said it would be "extremely difficult" to achieve the government\'s pre-protest forecast of 0% to 1% annual economic growth. Protests in the former British colony have reached their 21st week. On Sunday, black-clad and masked demonstrators set fire to shops and hurled petrol bombs at police who responded with tear gas, water cannon and rubber bullets. Protesters have routinely torched store fronts and businesses including banks, particularly those owned by mainland Chinese companies and vandalized t

The cluster was the assigment give by the Kmeans Process and the Sentence Transformer Process. The hierarchy was assigned by the Hierarchical Clustering using some centroid of the Kmenas. The category and the Topic name was the result process of using the top 20 words inside the Cluster in order to create the labels. the labels with be the category and the topic name.

Given that in the GLG process we are expecting to receive a paragrap with 3 to 6 sentences we will have to split the new and use only the first 5 sentences

In [8]:
sent_tokenize(dataset["train"][900]["paragraph"])[:3]

["Of the many major announcements Google made at its annual developer conference last week, from the debut of a connected jacket, to a smart home device to rival Amazon's Echo, one overshadowed them all: Daydream.",
 "Daydream, which will launch this fall as part of the newest version of Android, is Google's first high-quality virtual reality platform for mobile phones.",
 "It lumps all your phone's VR content into one app."]

In [9]:
def nltk_sent_tokenize(batch):
    """Tokenize a batch of text using nltk.sent_tokenize
    Args:
        batch (Dataset): Dataset dictionary with a "paragraphs" key
    Returns:
        dict: dictionary with a "sentences" key
    """
    sents = [ ' '.join(sent_tokenize(paragraph)[:5]) for paragraph in batch["paragraph"]]
    return {"sentences":sents}

In [10]:
dataset = dataset.map(nltk_sent_tokenize,batched=True,num_proc=8)

#0:   0%|          | 0/310 [00:00<?, ?ba/s]

#1:   0%|          | 0/310 [00:00<?, ?ba/s]

#2:   0%|          | 0/310 [00:00<?, ?ba/s]

#3:   0%|          | 0/310 [00:00<?, ?ba/s]

#4:   0%|          | 0/310 [00:00<?, ?ba/s]

#6:   0%|          | 0/310 [00:00<?, ?ba/s]

#5:   0%|          | 0/310 [00:00<?, ?ba/s]

#7:   0%|          | 0/310 [00:00<?, ?ba/s]

Example of the before and after sentence tokenizer

In [11]:
dataset["train"][900]["paragraph"]

"Of the many major announcements Google made at its annual developer conference last week, from the debut of a connected jacket, to a smart home device to rival Amazon's Echo, one overshadowed them all: Daydream. Daydream, which will launch this fall as part of the newest version of Android, is Google's first high-quality virtual reality platform for mobile phones. It lumps all your phone's VR content into one app. You'll still need to wear a headset, and use a small remote control to navigate your virtual world, but Daydream should make interacting with apps in a virtual space feel as easy and natural as sending a text. YouTube, Google Photos, Street View, and other, non-Google apps, such as Netflix and Hulu, are being rebuilt for the platform — this isn't just something niche or exclusive to gamers. But that's the (near) future. There's already a lot of cool VR content you can enjoy today — even Coachella got in on the action. To do that, though, you still need a headset. Luckily, th

In [12]:
dataset["train"][900]["sentences"]

"Of the many major announcements Google made at its annual developer conference last week, from the debut of a connected jacket, to a smart home device to rival Amazon's Echo, one overshadowed them all: Daydream. Daydream, which will launch this fall as part of the newest version of Android, is Google's first high-quality virtual reality platform for mobile phones. It lumps all your phone's VR content into one app. You'll still need to wear a headset, and use a small remote control to navigate your virtual world, but Daydream should make interacting with apps in a virtual space feel as easy and natural as sending a text. YouTube, Google Photos, Street View, and other, non-Google apps, such as Netflix and Hulu, are being rebuilt for the platform — this isn't just something niche or exclusive to gamers."

# Data preprocessing

For the Text Classification task it will be use a DistilBERT model. Given that is almost the Half of a BERT model and it mantains almost the same accuracy. The model will be the correlation between the `sentences` and the `labels`.

In [13]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer,pipeline

In [14]:
model_checkpoint = "distilbert-base-uncased"

In [15]:
tokenizer = DistilBertTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

to Use distilBERT model it will be use the DistilBERT pre-trained model. the tokenizer process will be in chare of assigning the vocabulary id with the CLS and SEP token, Truncate the text if necesary and to create the mask id for the model. The padding of the sentences will be handle by the DataColletor class. This allow to have a improvement on the speed of training.

Assign the labels as categorical variable.

In [16]:
#dataset = dataset.rename_column("labels","target")

In [17]:
dataset = dataset.class_encode_column("labels")

Casting to class labels:   0%|          | 0/2476 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/248 [00:00<?, ?ba/s]

Separate in train, validation and test set the data.

In [18]:
dataset = dataset.remove_columns(['paragraph', 'cluster', 'hierarchy', 'category', 'topic'])

In [20]:
dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'sentences'],
        num_rows: 2475520
    })
})

In [27]:
#dataset.save_to_disk("dataset")
dataset = load_from_disk("dataset")

In [28]:
dataset = dataset["train"].train_test_split(test_size=0.6,seed=794,stratify_by_column="labels")

In [29]:
dataset = dataset["train"].train_test_split(test_size=0.15,seed=19,stratify_by_column="labels")

In [30]:
test_validation_dataset = dataset["test"].train_test_split(test_size=0.15,seed=19,stratify_by_column="labels")

In [31]:
dataset["validation"] = test_validation_dataset["train"]
dataset["test"] = test_validation_dataset["test"]

In [32]:
dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'sentences'],
        num_rows: 841676
    })
    test: Dataset({
        features: ['labels', 'sentences'],
        num_rows: 22280
    })
    validation: Dataset({
        features: ['labels', 'sentences'],
        num_rows: 126252
    })
})

Tokenizer process before the model training

In [33]:
MAX_LEN = 512
def preprocess_function(batch):
    return tokenizer(batch['sentences'], truncation=True, max_length=MAX_LEN)

In [34]:
tokenized_dataset = dataset.map(
    preprocess_function
    ,batched=True
    ,num_proc=8
    ,remove_columns=['sentences']
)

#1:   0%|          | 0/106 [00:00<?, ?ba/s]

#2:   0%|          | 0/106 [00:00<?, ?ba/s]

#0:   0%|          | 0/106 [00:00<?, ?ba/s]

#3:   0%|          | 0/106 [00:00<?, ?ba/s]

#4:   0%|          | 0/106 [00:00<?, ?ba/s]

...

#3:   0%|          | 0/16 [00:00<?, ?ba/s]

#4:   0%|          | 0/16 [00:00<?, ?ba/s]

#7:   0%|          | 0/16 [00:00<?, ?ba/s]

#6:   0%|          | 0/16 [00:00<?, ?ba/s]

#5:   0%|          | 0/16 [00:00<?, ?ba/s]

Preparation of the data for the model training

In [35]:
#273
target_feature = tokenized_dataset["train"].features["labels"]

In [36]:
num_classes = target_feature.num_classes

In [37]:
label_names = target_feature.names
id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [38]:
model = DistilBertForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_classes,id2label=id2label,label2id=label2id)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier

In [39]:
for i,p in enumerate(model.distilbert.parameters()):
    if i < 95:
        p.requires_grad = False

In [40]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [41]:
batch_size = 448
model_name = "distilbert-topic-classification-chk"
warmup_steps=10
epochs = 3
logging_steps = len(tokenized_dataset["train"])// batch_size

training_args = TrainingArguments(
    output_dir="./{}".format(model_name),
    logging_dir='./logs',
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=0.1,
    weight_decay=0.005,
    warmup_steps=warmup_steps,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    logging_steps=logging_steps,
    report_to="all",
    optim="adafactor"
)

In [42]:
from sklearn import metrics

In [43]:
def compute_metrics(eval_pred):
    logits_, labels_ = eval_pred
    predictions = np.argmax(logits_, axis=-1)

    accuracy = metrics.accuracy_score(labels_, predictions)
    f1_score_micro = metrics.f1_score(labels_, predictions, average='micro')
    f1_score_macro = metrics.f1_score(labels_, predictions, average='macro')

    return {"accuracy": accuracy, "f1_score_micro": f1_score_micro, "f1_score_macro": f1_score_macro}



In [44]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)
trainer.train()

***** Running training *****
  Num examples = 841676
  Num Epochs = 3
  Instantaneous batch size per device = 448
  Total train batch size (w. parallel, distributed & accumulation) = 448
  Gradient Accumulation steps = 1
  Total optimization steps = 5637


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score Micro,F1 Score Macro
1,5.056900,4.580123,0.158580,0.158580,0.001003


***** Running Evaluation *****
  Num examples = 126252
  Batch size = 448
Saving model checkpoint to ./distilbert-topic-classification-chk/checkpoint-1879
Configuration saved in ./distilbert-topic-classification-chk/checkpoint-1879/config.json
Model weights saved in ./distilbert-topic-classification-chk/checkpoint-1879/pytorch_model.bin
tokenizer config file saved in ./distilbert-topic-classification-chk/checkpoint-1879/tokenizer_config.json
Special tokens file saved in ./distilbert-topic-classification-chk/checkpoint-1879/special_tokens_map.json


KeyboardInterrupt: 

In [ ]:
trainer.save_model("distilbert-topic-classification")

AttributeError: 'Trainer' object has no attribute 'save_pretrained'